In [1]:
# Needed for static or interactive matplotlib plots
%matplotlib inline

import sys
from pprint import pprint as pp

import concurrent.futures

from scipy.signal import convolve2d
from scipy.sparse import diags, csr_matrix, bsr_matrix, spdiags, lil_matrix, dok_matrix
from scipy.sparse.linalg import aslinearoperator

from skimage import data, io, color
from skimage.transform import resize

from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5, 5]

# Sneaky hack to install dask, sparse
!{sys.executable} -m pip install "dask[complete]" sparse

import dask
import dask.array as da
from dask.distributed import Client
import sparse


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
def matrix_memory(m):
    return (m.data.nbytes + m.indptr.nbytes + m.indices.nbytes) / 10**6 # MB

def matrix_density(m):
    return m.getnnz() / np.prod(m.shape)

def show_image(image, title):
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.title(title)
    plt.colorbar()
    plt.show()
    
def show_sparse_matrix(image, title):
    plt.spy(image)
    plt.title(title)
    plt.show()
    
def gaussian(l=5, sig=1.):
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

def normalise(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data)) 

### Trying to break up matrix mult into stackable partitions

In [3]:
def generate_h_matrix(l, kernel):
    h = conv_matrix(l, kernel)
    print(f"H matrix memory usage: {matrix_memory(h)}MB")
    print(f"H matrix density: {matrix_density(h)}%")
    return h 

def conv_matrix(l, kernel):
    k_supp = kernel.shape[0]
    k_half_supp = (k_supp-1)//2
    k_samples = k_supp**2
    col_offsets = np.repeat(np.arange(k_supp) - k_half_supp, k_supp) * (l - k_supp)
    diagonal_offsets = (np.arange(k_samples) - (k_samples-1)//2) + col_offsets
    m = diags(kernel.flatten(), diagonal_offsets, shape=(l**2, l**2), format="csr", dtype=np.float32)
    
    mask_vals = np.repeat(1.0, k_supp)
    mask_offsets = np.linspace(-k_half_supp, k_half_supp, k_supp, dtype=np.intc)
    mask = diags(mask_vals, mask_offsets, shape=(l, l), format="bsr", dtype=np.float32)
    
    try:
        Parallel(n_jobs=-1, prefer='threads')(delayed(apply_mask)(row, l, k_half_supp, m, mask) for row in range(l))
    except Exception as e:
        print("Error...")
        print(e)
        
    return m  

def apply_mask(r, l, k_half_supp, m, mask):
    col_chunk_min = max(0, r-k_half_supp-1)
    col_chunk_max = min(l-1, r+k_half_supp+1)
    
    for c in range(col_chunk_min, col_chunk_max+1):
        m[r*l:r*l+l, c*l:c*l+l] = m[r*l:r*l+l, c*l:c*l+l].multiply(mask)
        
def decimation_matrix(original_dim, downsample_factor):
    
    if original_dim % downsample_factor != 0:
        raise ValueError(f"Downsample factor {downsample_factor} is not a valid factor of your matrix dimension {original_dim}.")
    if downsample_factor == original_dim:
        raise ValueError(f"Downsample factor {downsample_factor} cannot be the same as your matrix dimension {original_dim}.")
    if downsample_factor == 1: # effectively, no downsampling
        return np.identity(original_dim**2)
    # Otherwise assumed you want to downsample by a valid factor of original_dim...
    
    sampling_regions_per_dim = original_dim // downsample_factor
    samples_per_region_dim = downsample_factor
    # print(f"Sampling regions per dimension: {sampling_regions_per_dim}")
    # print(f"Samples per dimension: {samples_per_region_dim}")
    non_zero_entries = sampling_regions_per_dim**2 * samples_per_region_dim**2
    # print(f"Non-zero entries: {non_zero_entries}")
    
    rows = np.zeros(non_zero_entries, dtype=np.uintc)   # stores row indices for non-zero compressed sparse matrix entries
    cols = np.zeros(non_zero_entries, dtype=np.uintc)   # stores col indices for non-zero compressed sparse matrix entries
    vals = np.ones(non_zero_entries, dtype=np.float32)  # stores element value at [row, col] for non-zero entries
    
    # Generates linear x,y index strides for downsampling
    sample_stride_1D = np.arange(0, original_dim, downsample_factor)
    # print(sample_stride_1D)
    mesh = np.array(np.meshgrid(sample_stride_1D, sample_stride_1D))
    sample_strides_2D = mesh.T.reshape(-1, 2)
  
    neighbour_strides_1D = np.arange(samples_per_region_dim)
    neighbour_mesh = np.array(np.meshgrid(neighbour_strides_1D, neighbour_strides_1D))

    for index in np.arange(sample_strides_2D.shape[0]):
        neighbour_coords = neighbour_mesh.T.reshape(-1, 2) + sample_strides_2D[index] # generates (row, col) index pair for the nxn neighbours of each sampling point in sample_strides_2D
        neighbour_coords[:, 0] *= original_dim # scale y coord by high-resolution image dim to enable row striding (due to column-vector matrix flattening)
        neighbour_coords = np.sum(neighbour_coords, axis=1) # combine x and y coord into single array index
        rows[index * neighbour_coords.shape[0] : (index + 1) * neighbour_coords.shape[0]] = index
        cols[index * neighbour_coords.shape[0] : (index + 1) * neighbour_coords.shape[0]] = neighbour_coords
        
    return (csr_matrix((vals, (rows, cols))))

In [4]:
l = 40
k_supp = 3
kernel = gaussian(k_supp)
df = 2

d = decimation_matrix(l, df)
# show_image(d.todense(), "D")

h = generate_h_matrix(l, kernel)
# show_image(h.todense(), "H")

dh_orig = d @ h
# show_image(dh.todense(), "DH")

H matrix memory usage: 0.119668MB
H matrix density: 0.00553046875%


In [5]:
# Distribution client for Dask
# client = Client()

In [9]:
# @dask.delayed
def derp(x, block_info=None):
    
    if block_info is not None:
        # print(x)
        # pp(block_info)
        chunk_y_range_global = block_info[None]['array-location'][0]
        chunk_x_range_global = block_info[None]['array-location'][1]
        chunk_shape = block_info[None]['chunk-shape']
        
        for row in range(chunk_shape[0]):
            chunk_y_index_global = chunk_y_range_global[0] + row
            for col in range(chunk_shape[1]):
                chunk_x_index_global = chunk_x_range_global[0] + col
                # print(f"Local row/col -> Global row/col: {row}, {col} => {chunk_y_index_global}, {chunk_x_index_global}")
                x[row, col] = (chunk_y_index_global * chunk_shape[0]) + chunk_x_index_global
                
    return x

height = 1000
width = 1000
chunk_y = 10
chunk_x = 10

# data = np.arange(height * width, dtype=np.uintc).reshape(height, width)
# a = da.from_array(data, chunks=(chunk_y, chunk_x))
# a = sqr(a)
# dict(a.__dask_graph__())
# %time a.compute()

x = da.zeros((height, width), chunks=(chunk_y, chunk_x))
x[x < 0.5] = 0
# print(x.compute())
s = x.map_blocks(sparse.DOK)
s = s.map_blocks(derp)
%time s = s.compute()
# s.todense()
# dhp = dok_matrix((d.shape[0], h.shape[1]), dtype=np.float32)

# chunk_dim = 2
# x = da.arange(chunk_dim**2, dtype=np.float32)
# x = da.reshape(x, (chunk_dim, chunk_dim))
# s = x.map_blocks(sparse.COO)
# s.compute()
# s = s.map_blocks(parallel_dh).compute()
# show_image(s.todense(), "S")

CPU times: user 45.6 s, sys: 676 ms, total: 46.3 s
Wall time: 45.4 s


In [7]:
# h_clip = lil_matrix((h.shape), dtype=np.float32)
# h_clip[:10, :10] = h[:10, :10]
# show_image(h_clip.todense(), "H Clip")

# dh = d[:10, :] @ h_clip
# show_image(dh.todense(), "DH Clip")

# h_clip = lil_matrix((h.shape), dtype=np.float32)
# h_clip[:10, 10:20] = h[:10, 10:20]
# show_image(h_clip.todense(), "H Clip")
# dh += d[0:10, :] @ h_clip
# show_image(dh.todense(), "DH Clip")